# 5장. &#129303;Datasets
- [강좌링크](https://wikidocs.net/166806)

In [1]:
# Notebook 초기화

from custom_utils import *

from datasets import Dataset, IterableDatasetDict, DownloadConfig, interleave_datasets, load_dataset, load_from_disk
from itertools import islice
from transformers import AutoModel, AutoTokenizer

import html
import psutil
import timeit

wrapper = CustomObject()

# 1. 데이터셋이 허브에 없다면?

## 로컬 혹은 원격 데이터셋으로 작업하기
|    Data Format     |  Loading script   | Example                                              |
|:------------------:|:-----------------:|:-----------------------------------------------------|
|     CSV & TSV      |        csv        | load_dataset("csv", data_files="myfile.csv")         |
|     Textfiles      |       text        | load_dataset("txt", data_files="myfile.txt")         |
| JSON & JSON Lines  |       json        | load_dataset("json", data_files="myfile.jsonl")      |
| Pickled DataFrames |      pandas       | load_dataset("pandas", data_files="myDataFrame.pkl") |

## 로컬 데이터셋 로딩하기
예제 데이터로 이탈리아어 질의응답을 위한 대규모 데이터셋인 **SQuAD-it dataset**을 사용

학습 및 평가 집합으로 분할된 파일들은 github에서 호스팅되므로 wget 명령으로 쉽게 다운로드 가능
```shell
cd ../datas
wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz
```
다운로드 후 압축 해제
```shell
gzip -dkv SQuAD_it*.json.gz
```

In [2]:
wrapper.squad_it_dataset = load_dataset("json", data_files = "../datas/SQuAD_it-train.json", field = "data")
print(wrapper.squad_it_dataset)
print(wrapper.squad_it_dataset["train"][2])
wrapper.init_wrapper()

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
})
{'title': '51º stato', 'paragraphs': [{'context': 'La frase "51° stato" può essere usata in senso positivo, nel senso che una regione o territorio è così allineata, solidale e favorevole agli Stati Uniti che è come uno stato americano. Può anche essere utilizzato in senso peggiorativo, il che significa che un\' area o una regione è percepita come sottoposta a un\' eccessiva influenza o controllo militare o culturale americano. In vari paesi del mondo, le persone che credono che la loro cultura locale o nazionale sia troppo americanizzata usano talvolta il termine "51° stato" in riferimento al proprio paese. eccessiva influenza o controllo culturale o militare americano eccessivo.', 'qas': [{'answers': [{'answer_start': 71, 'text': 'una regione o territorio è così allineata, solidale e favorevole agli Stati Uniti'}], 'id': '572eedffc246551400ce47bc', 'question': 'Qual è la conno

`DatasetDict` 객체가 생성되었으나 학습 분할된 두 파일을 모두 포함시켜 한번에 두 집합에 `Dataset.map()`함수를 적용시키고 싶다.

이를 위해 `data_files` 매개변수에 각 분할 이름을 해당 집합 파일명에 매핑하는 딕셔너리를 지정하면 된다.

In [3]:
wrapper.data_files = {
    "train": "../datas/SQuAD_it-train.json",
    "test": "../datas/SQuAD_it-test.json",
    
    # 자동 압축해제 가능: gz, tar, zip 등
    # "train": "../datas/SQuAD_it-train.json.gz",
    # "test": "../datas/SQuAD_it-test.json.gz"
}
wrapper.squad_it_dataset = load_dataset("json", data_files = wrapper.data_files, field = "data")
wrapper.squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

## 원격 데이터셋 로딩하기

로컬 데이터셋 로딩과 똑같지만 data_files에 url을 입력한다.

In [4]:
wrapper.url = "https://github.com/crux82/squad-it/raw/master/"
wrapper.data_files = {
    "train": wrapper.url + "SQuAD_it-train.json.gz",
    "test": wrapper.url + "SQuAD_it-test.json.gz"
}
wrapper.squad_it_dataset = load_dataset("json", data_files = wrapper.data_files, field = "data")
print(wrapper.squad_it_dataset)

wrapper.init_wrapper()

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

GPU NVIDIA GeForce RTX 3060
memory occupied: 0.81GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


# 2. Dataset Slicing and Dicing

&#129303;Datasets는 Pandas와 유사하게 `Dataset` 및 `DatasetDict` 객체의 내용을 조작하는 여러 기능을 제공한다.

이번 예제는 UC Irvine Machine Learning Repository에서 호스팅되는 Drug Review Dataset을 사용하는데 이는 다양한 약물에 대한 환자 리뷰, 치료 상태 및 환자 만족도에 대한 별 10개 등급이 포함되어있다.

```shell
cd ../datas
wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
unzip drugsCom_raw.zip
```

쉼표 대신 탭문자를 쓰는 CSV인 TSV 파일이므로 parameter에 `delimeter = "\t"`를 추가한다.

&#129303;Datasets의 `shuffle()`과 `select()` 함수를 연결해 무작위 샘플을 가져올 수 있다.

In [5]:
wrapper.data_files = {
    "train": "../datas/drugsComTrain_raw.tsv",
    "test": "../datas/drugsComTest_raw.tsv"
}
wrapper.drug_dataset = load_dataset("csv", data_files = wrapper.data_files, delimiter = "\t")
wrapper.drug_sample = wrapper.drug_dataset["train"].shuffle(seed = 42).select(range(1000))
wrapper.drug_sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

해당 dataset의 단점(quirks) 몇가지를 나열하자면:
1. `"Unnamed: 0"` column은 아마 각 환자의 익명 ID로 추측됨
2. `"condition"` column은 대문자와 소문자 레이블이 혼합되어 있음
3. 리뷰의 길이는 다양하며 Python 줄 구분 기호(\r\n)와 같은 HTML 문자 코드가 혼합되어 있음.

> &#129303;Datasets를 사용하여 이런 문제를 처리할 수 있다.

Unnamed:0 column이 환자의 ID가 맞는지 Dataset.unique() 함수를 사용해 확인할 수 있다.

ID 값이 맞다면 `DatasetDict.rename_column()` 함수를 사용해 한 번에 두 분할에서 컬럼명을 변경할 수 있다.

In [6]:
for split in wrapper.drug_dataset.keys():
    assert len(wrapper.drug_dataset[split]) == len(wrapper.drug_dataset[split].unique("Unnamed: 0"))

wrapper.drug_dataset = wrapper.drug_dataset.rename_column(
    original_column_name = "Unnamed: 0",
    new_column_name = "patient_id"
)
wrapper.drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

다음으로 `Dataset.map()`을 사용하여 None값이 아닌 행을 제외한 데이터셋의 모든 `condition` 레이블을 정규화한다.

In [7]:
def lowercase_condition(example):
    return {"condition": example["condition"].lower()}

wrapper.drug_dataset = wrapper.drug_dataset.filter(lambda x: x["condition"] is not None)
wrapper.drug_dataset.map(lowercase_condition)

# 확인
wrapper.drug_dataset["train"][:3]

{'patient_id': [206461, 95260, 92703],
 'drugName': ['Valsartan', 'Guanfacine', 'Lybrel'],
 'condition': ['Left Ventricular Dysfunction', 'ADHD', 'Birth Control'],
 'review': ['"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
  '"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effect

## 새로운 column 만들기

각 리뷰의 단어 수를 계산하기 위해 각 텍스트를 공백으로 나누는 대략적인 heuristic을 사용한다.

리뷰의 단어 수로 정렬하여 가장 적은 단어를 사용한 리뷰를 확인할 수 있고 30단어 미만으로 표현된 리뷰를 제거할 수 있다.

In [8]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}

wrapper.drug_dataset = wrapper.drug_dataset.map(compute_review_length)
print(wrapper.drug_dataset["train"].sort("review_length")[:3])

"""
또는 아래와 같이 추가하는 방법도 있다.
for split in wrapper.drug_dataset.keys():
    wrapper.drug_dataset[split].add_column("review_length", [len(a.split()) for a in wrapper.drug_dataset[split]["review"]])
"""

wrapper.drug_dataset = wrapper.drug_dataset.filter(lambda x: x["review_length"] > 30)
print(wrapper.drug_dataset.num_rows)

{'patient_id': [111469, 13653, 53602], 'drugName': ['Ledipasvir / sofosbuvir', 'Amphetamine / dextroamphetamine', 'Alesse'], 'condition': ['Hepatitis C', 'ADHD', 'Birth Control'], 'review': ['"Headache"', '"Great"', '"Awesome"'], 'rating': [10.0, 10.0, 10.0], 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'], 'usefulCount': [41, 3, 0], 'review_length': [1, 1, 1]}
{'train': 138514, 'test': 46108}


## HTML 문자 코드 제거

Python의 `HTML` 모듈을 사용해 해결 가능

In [9]:
wrapper.text = "I&#039;m a transformer called BERT"
html.unescape(wrapper.text)

"I'm a transformer called BERT"

In [10]:
wrapper.drug_dataset = wrapper.drug_dataset.map(lambda x: {"review": html.unescape(x["review"])})

## map()의 다양한 능력

- `batched`: 매개변수, True로 설정되면 batch가 한 번에 `map()`에 입력됨, default = 1000
    위의 unescape map의 실행 속도는 약간 느린데, list comprehension을 사용해 동시에 여러 예제를 한 번에 처리하여 속도를 높일 수 있다.
    Inputs: List, Outputs: List

In [11]:
wrapper.new_drug_dataset = wrapper.drug_dataset.map(
    lambda x: {"review": [html.unescape(o) for o in x["review"]]},
    batched = True
)

`Dataset.map()`을 batch와 함께 사용하는 것은 fast Tokenizer의 속도를 unlock하는데 필수이며 이는 규모가 큰 텍스트 리스트를 빠르게 토큰화할 수 있다.

In [12]:
wrapper.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
wrapper.slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast = False)

def tokenize_function(examples, is_fast = True):
    if is_fast:
        return wrapper.tokenizer(examples["review"], truncation = True)
    return wrapper.slow_tokenizer(examples["review"], truncation = True)

# slow tokenizer
print("slow tokenizer")
%time wrapper.drug_dataset.map(lambda x: tokenize_function(x, False))

# slow tokenizer batched
print("slow tokenizer batched")
%time wrapper.drug_dataset.map(lambda x: tokenize_function(x, False), batched = True, num_proc = 8)

# fast tokenizer
print("fast tokenizer")
%time wrapper.drug_dataset.map(lambda x: tokenize_function(x))

# fast tokenizer batched
print("fast tokenizer batched")
%time wrapper.drug_dataset.map(lambda x: tokenize_function(x), batched = True, num_proc = 8)

slow tokenizer
CPU times: user 251 ms, sys: 2.02 ms, total: 253 ms
Wall time: 253 ms
slow tokenizer batched
CPU times: user 278 ms, sys: 0 ns, total: 278 ms
Wall time: 278 ms
fast tokenizer


Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 11.3 s, sys: 54.5 ms, total: 11.3 s
Wall time: 11.3 s
fast tokenizer batched
CPU times: user 273 ms, sys: 1.08 ms, total: 274 ms
Wall time: 274 ms


DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 138514
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 46108
    })
})

AutoTokenizer의 default가 fast tokenizer인 이유가 보임 거의 40배는 빨라짐

매개변수 `num_proc`의 경우 8로 했을 때가 가장 빠르다고 함. 그 외에는 `num_proc` 값을 지정하지 않은 경우가 더 빠름.

__일반적으로__ `batched = True`인 fast Tokenizer에는 Python 다중처리를 하지 않는 것이 좋다고 함.

이 모든 기능을 하나로 합친게 `Dataset.map()`

일반적으로 기계 학습에서 하나의 예제는 모델에 제공하는 feature의 집합으로 정의된다. 특정 상황에서 이런 feature는 `Dataset` 내의 column으로 표현되지만 다른 맥락에서는 여러개의 feature가 하나의 단일 예제에서 추출되어 column에 속하는 경우도 있다.

아래 코드는 예제(example)들을 토큰화하고 최대 길이 128로 자른다. 그러나 이 과정에서 tokenizer에게 전체 `review`의 앞부분 128개 토큰으로 구성된 chunk가 아니라 텍스트의 모든 chunk를 반환(`return_overflowing_tokens=True`)하도록 요청한다.

In [13]:
def tokenize_and_split(examples):
    return wrapper.tokenizer(
        examples["review"],
        truncation = True,
        max_length = 128,
        return_overflowing_tokens = True
    )

# 이전 데이터셋의 열을 제거하거나 새로운 데이터셋과 동일한 크기로 만들며 수행(remove_columns)
wrapper.tokenized_dataset = wrapper.drug_dataset.map(
    tokenize_and_split,
    batched = True,
    remove_columns = wrapper.drug_dataset["train"].column_names
)

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [14]:
print(len(wrapper.tokenized_dataset["train"]), len(wrapper.drug_dataset["train"]))
wrapper.tokenized_dataset

204198 138514


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 204198
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 68023
    })
})

이전 열을 새 열과 같은 크기로 만들어 길이 불일치 문제를 해결할 수도 있다고 했는데 이를 위해 `return_overflowing_tokens=True`설정 시 Tokenizer가 반환하는 `overflow_to_sample_mapping` 필드가 필요하다. 이 필드는 새로운 feature index에서 샘플의 인덱스로의 매핑을 제공한다. 이를 사용해 새 feature를 생성하는 횟수만큼 각 예제의 값을 반복해 원본 데이터셋에 있는 각 키를 올바른 크기의 값 목록과 연결할 수 있다.

In [15]:
def tokenize_and_split(examples):
    result = wrapper.tokenizer(
        examples["review"],
        truncation = True,
        max_length = 128,
        return_overflowing_tokens = True
    )
    
    # 신규 인덱스와 이전 인덱스와의 매핑 추출
    sample_map = result.pop("overflow_to_sample_mapping")
    for k, v in examples.items():
        result[k] = [v[i] for i in sample_map]
    return result

# 이로써 이전 열을 제거할 필요 없이 `Dataset.map()`과 함께 작동
wrapper.tokenized_dataset = wrapper.drug_dataset.map(tokenize_and_split, batched = True)

# 위와 동일하게 증가된 수의 학습 feature를 얻었지만 이전 필드를 모두 유지한다.
wrapper.tokenized_dataset

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 204198
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 68023
    })
})

&#129303;Datasets를 사용하여 다양한 방식으로 데이터셋을 preprocessing하는 법을 확인했다. &#129303;Datasets의 함수들은 모델 학습에 필요한 대부분의 요구 사항을 수용하지만 `DataFrame.groupby()` 또는 시각화를 위한 고급 API와 같은 보다 강력한 기능을 사용하기 위해 Pandas를 사용해야 하는 경우가 있다.

다행히 &#129303;Datasets는 Pandas, NumPy, PyTorch, TensorFlow, JAX 등과 같은 라이브러리와 상호 운용 가능하도록 설계되었다.

## Datasets과 DataFrames 간의 상호 변환

다양한 서드파티 라이브러리들 간의 변환을 가능하게 하기 위해 &#129303;Datasets는 `Dataset.set_format()` 함수를 제공해 기본 데이터 포맷(Apache Arrow)에 영향을 주지 않고 데이터셋의 출력 형식만 쉽게 전환할 수 있다.

In [16]:
"""
내부적으로 __getitem__()의 반환 형식을 변경하기 때문에
wrapper.drug_dataset["train"].head(3)와 같은 문법은 사용 불가능하다.
Dataset.head()는 없기 때문
> set_format("pandas")으로 DataFrame을 얻으려면 slicing해야함.
"""
wrapper.drug_dataset.set_format("pandas")
print(wrapper.drug_dataset["train"][:3])
print(wrapper.drug_dataset["train"][:].head(2))

wrapper.train_df = wrapper.drug_dataset["train"][:]

   patient_id    drugName      condition  \
0       95260  Guanfacine           ADHD   
1       92703      Lybrel  Birth Control   
2      138000  Ortho Evra  Birth Control   

                                              review  rating  \
0  "My son is halfway through his fourth week of ...     8.0   
1  "I used to take another oral contraceptive, wh...     5.0   
2  "This is my first time using any form of birth...     8.0   

                date  usefulCount  review_length  
0     April 27, 2010          192            141  
1  December 14, 2009           17            134  
2   November 3, 2015           10             89  
   patient_id    drugName      condition  \
0       95260  Guanfacine           ADHD   
1       92703      Lybrel  Birth Control   

                                              review  rating  \
0  "My son is halfway through his fourth week of ...     8.0   
1  "I used to take another oral contraceptive, wh...     5.0   

                date  usefulCount  r

Dataset을 slicing해 DataFrame으로 만들었으니 모든 Pandas 기능을 사용할 수 있다.

예를 들어, `condition` 항목의 클래스 분포를 계산하기 위한 chaining하거나 새로운 `Dataset` 객체를 생성할 수 있다.

In [18]:
wrapper.frequencies = (
    wrapper.train_df["condition"]
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns = {
        "index": "condition",
        "condition": "frequency"
    })
)
wrapper.frequencies.head()

,frequency,count
0,Birth Control,27655
1,Depression,8023
2,Acne,5209
3,Anxiety,4991
4,Pain,4744


In [21]:
# 복구
wrapper.drug_dataset.reset_format()

# Dataset from Pandas
wrapper.freq_dataset = Dataset.from_pandas(wrapper.frequencies)
wrapper.freq_dataset

Dataset({
    features: ['frequency', 'count'],
    num_rows: 819
})

## Validation Set 생성

모델 성능 평가에 사용할 수 있는 test set이 있지만 학습 성능 향상을 위해 개발 중에 별도의 검증 집합을 만드는 것이 좋다.

검증 집합에서 모델의 성능에 만족하면 test set에서 최종적인 온전성 검사 sanity check를 수행할 수 있다.

&#129303;Datasets는 `scikit-learn`의 기능을 기반으로 하는 `Dataset.train_test_split()` 함수를 제공한다.

In [22]:
# train, test로 나뉘어진 Dataset 반환
wrapper.drug_dataset_clean = wrapper.drug_dataset["train"].train_test_split(train_size = 0.8, seed = 42)

# test Dataset을 validation으로 변경
wrapper.drug_dataset_clean["validation"] = wrapper.drug_dataset_clean.pop("test")

# DatasetDict에 test Dataset을 추가
wrapper.drug_dataset_clean["test"] = wrapper.drug_dataset["test"]

wrapper.drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

## Dataset 저장

&#129303;Datasets는 다운로드한 모든 데이터셋과 수행된 작업을 임시저장하지만 디스크에 저장하고 싶을 때가 있는데 아래와 같이 저장한다.

|  Data format  | Function               |
|:-------------:|------------------------|
|     Arrow     | Dataset.save_to_disk() |
|      CSV      | Dataset.to_csv()       |
|     JSON      | Dataset.to_json()      |
![](../assets/drug_reviews_tree.png)

In [25]:
wrapper.drug_dataset_clean.save_to_disk("../datas/drug_reviews")

wrapper.drug_dataset_reload = load_from_disk("../datas/drug_reviews")
print(wrapper.drug_dataset_reload)

del lowercase_condition, compute_review_length, tokenize_function, tokenize_and_split
wrapper.init_wrapper()

Saving the dataset (0/1 shards):   0%|          | 0/110811 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27703 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/46108 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

GPU NVIDIA GeForce RTX 3060
memory occupied: 0.81GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


CSV 및 JSON 형식은 각 분할 { train, test, validation }을 별도의 파일로 저장해야 한다.
```python
for split, dataset in wrapper.drug_dataset_clean.items():
    dataset.to_json(f"drug-reviews-{split}.jsonl") # 형식은 csv, json, jsonl 등 사용 가능함

# 다시 불러올 때는 위에서처럼 load_dataset() 사용
```

# 3. &#129303;Datasets가 빅데이터 문제를 해결한다.

데이터셋을 memory-mapped 파일로 처리하여 메모리 관리 문제를 해결하고 corpus의 각 항목들을 스트리밍하여 하드디스크 제한에서 해방한다.

이 섹션에서는 Pile로 알려진 825GB 규모의 데이터셋을 이용한다.

## Pile
EleutherAI가 대규모 언어 모델을 학습하기 위해 만든 영어 텍스트 말뭉치로, 학술 논문, github 코드 리포지토리 및 필터링된 웹 텍스트에 이르는 다양한 데이터셋이 포함되어 있다. 학습 데이터는 14GB 청크로 제공되며 개별적인 구성 요소를 각각 다운로드할 수 있다. 먼저, 1,500만 건으로 구성된 생의학 출판물 초록 모음인 PubMed Abstracts 데이터셋을 보면 JSON Lines 형식이고 `zstandard` 라이브러리를 사용하여 압축되므로 먼저 zstandard를 설치해야 한다.

In [6]:
# url 오류로 [토렌트 파일 다운](https://the-eye.eu/public/AI/EleutherAI_ThePile_v1.torrent)받아서 로컬에서 불러옴 PUBMED_title_abstracts_2019_baseline.jsonl.zst만 다운받으면 6.42GB
# wrapper.data_files = "https://the-eye.eu/public/AI/pile_preliminary_components/PUBMED_title_abstracts_2019_baseline.jsonl.zst"
wrapper.data_files = "../datas/EleutherAI_ThePile_v1/pile_preliminary_components/PUBMED_title_abstracts_2019_baseline.jsonl.zst"

# 하드디스크의 공간을 절약하기 위해 download_config 인수에 DownloadConfig(delete_extracted=True) 전달하여 압축해제된 파일 제거
wrapper.pubmed_dataset = load_dataset("json", data_files = wrapper.data_files, split = "train", download_config = DownloadConfig(delete_extracted = True))

print(wrapper.pubmed_dataset)
print(wrapper.pubmed_dataset[0])

Dataset({
    features: ['meta', 'text'],
    num_rows: 15518009
})
{'meta': {'pmid': 11409574, 'language': 'eng'}, 'text': 'Epidemiology of hypoxaemia in children with acute lower respiratory infection.\nTo determine the prevalence of hypoxaemia in children aged under 5 years suffering acute lower respiratory infections (ALRI), the risk factors for hypoxaemia in children under 5 years of age with ALRI, and the association of hypoxaemia with an increased risk of dying in children of the same age. Systematic review of the published literature. Out-patient clinics, emergency departments and hospitalisation wards in 23 health centres from 10 countries. Cohort studies reporting the frequency of hypoxaemia in children under 5 years of age with ALRI, and the association between hypoxaemia and the risk of dying. Prevalence of hypoxaemia measured in children with ARI and relative risks for the association between the severity of illness and the frequency of hypoxaemia, and between hypoxaemia a

## memory-mapping

메모리 사용량을 측정하기 위해 process util에 관련된 psutil 라이브러리 사용

In [7]:
# rss 속성은 프로세스가 RAM에서 차지하는 메모리 비율인 `Resident Set Size`를 의미.
print(f"RAM used: {psutil.Process().memory_info().rss / (1024**2):.2f} MB")

# `dataset_size` 속성을 사용해 데이터셋이 디스크에서 어느 정도 크기인지 확인
print(f"Number of files is dataset: {wrapper.pubmed_dataset.dataset_size}")
wrapper.size_gb = wrapper.pubmed_dataset.dataset_size / (1024 ** 3)
print(f"Dataset size (cache file): {wrapper.size_gb:.2f}GB")

RAM used: 1325.50 MB
Number of files is dataset: 20978892555
Dataset size (cache file): 19.54GB


&#129303;Datasets는 각 데이터셋을 memory-mapped file로 처리한다. 이 파일은 RAM과 파일 시스템 스토리지 간의 매핑을 제공하여 라이브러리가 데이터셋을 메모리에 완전히 로드할 필요없이 각 요소에 액세스하고 작동할 수 있도록 해준다.

memory-mapped file은 여러 프로세스에서 공유될 수 있으므로 `Dataset.map()`과 같은 메서드를 병렬화할 수 있다.

In [9]:
# Python의 timeit 모듈을 사용하여 batch_size가 1000일 때 데이터셋 조회 실행 시간을 측정

wrapper.code_snippet = """wrapper.batch_size = 1000

for idx in range(0, len(wrapper.pubmed_dataset), wrapper.batch_size):
    _ = wrapper.pubmed_dataset[idx:idx + wrapper.batch_size]
"""

wrapper.time = timeit.timeit(stmt = wrapper.code_snippet, number = 1, globals = globals())
print(f"Iterated over {len(wrapper.pubmed_dataset)} examples (about {wrapper.size_gb:.1f} GB in {wrapper.time:.1f}s, i.e. {wrapper.size_gb / wrapper.time:.3f} GB/s")

wrapper.pubmed_dataset.cleanup_cache_files()

wrapper.init_wrapper()

Iterated over 15518009 examples (about 19.5 GB in 50.3s, i.e. 0.389 GB/s

GPU NVIDIA GeForce RTX 3060
memory occupied: 1.16GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


## Streaming Datasets

&#129303;Datasets는 전체 데이터셋을 다운로드할 필요없이 즉시 요소를 다운로드하고 액세스할 수 있는 스트리밍 기능을 제공한다.

방법은 `streaming=True` 인수를 `load_dataset()`에 전달하기만 하면 된다.

이때 반환되는 객체는 Dataset이 아니라 `IterableDataset`이다. 스트리밍된 데이터셋의 요소는 `IterableDataset.map()`을 사용하여 바로 처리할 수 있지만 요소에 액세스하려면 `next(iter(dataset))`과 같이 반복해야 한다.

`IterableDataset.shuffle()`을 사용해 셔플링할 수도 있지만 `Dataset.shuffle()`과 달리 사전 정의된 `buffer_size` 인수만큼의 요소들만 먼저 셔플링한다. 반환된 IterableDataset의 다음값을 출력하면 버퍼가 다음 예제로 채워지고 셔플링한다.

`IterableDataset.skip(n)`으로 처음 n개의 예제를 제외한 나머지 예제를 선택할 수 있고 `IterableDataset.take(n)`으로 처음 n개의 예제를 선택할 수 있다.

또한 &#129303;Datasets는 IterableDataset 리스트를 하나의 IterableDataset으로 변환하는 `interleave_datasets()`함수를 제공한다.

In [26]:
wrapper.base_url = "https://the-eye.eu/public/AI/pile"
wrapper.data_files = {
    "train": [f"{wrapper.base_url}/train/{idx:02d}.jsonl.zst" for idx in range(30)],
    "validation": f"{wrapper.base_url}/val.jsonl.zst",
    "test": f"{wrapper.base_url}/test.jsonl.zst"
}
wrapper.pile_dataset = load_dataset("json", data_files = wrapper.data_files, streaming = True)
print(next(iter(wrapper.pile_dataset["train"])))
print(wrapper.pile_dataset)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

{'text': 'It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.\n\nThere’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.\n\nConcept\n\nWorking over the theme was probably one of the hardest tasks I had to face.\n\nOriginally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.\n\nIn the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a species facing a challenge.

In [27]:
# shuffle
wrapper.buffer_size = 1000
wrapper.shuffled_pile_dataset = wrapper.pile_dataset.shuffle(buffer_size = wrapper.buffer_size, seed = 42)

def compare(shuffled_data: IterableDatasetDict, pile_data: IterableDatasetDict) -> bool:
    if shuffled_data["text"] != pile_data["text"]:
        return False
    
    features1 = sorted(shuffled_data["meta"].features.items(), key = lambda x: x[0])
    features2 = sorted(pile_data["meta"].features.items(), key = lambda x: x[0])
    
    for f1, f2 in zip(features1, features2):
        if f1 != f2:
            return False
    
    return True

if compare(next(iter(wrapper.shuffled_pile_dataset["train"].skip(wrapper.buffer_size))),
           next(iter(wrapper.pile_dataset["train"].skip(wrapper.buffer_size)))):
    print("shuffle 후 next(iter()) 값이 셔플되지 않음")
else:
    print("shuffle 후 next(iter()) 값이 셔플됨")

shuffle 후 next(iter()) 값이 셔플됨


In [28]:
wrapper.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# default batch_size = 1000 if batched=True
wrapper.tokenized_dataset = wrapper.pile_dataset["train"].map(lambda x: wrapper.tokenizer(x["text"]), batched = True, batch_size = 10000)
next(iter(wrapper.tokenized_dataset)).keys()

Token indices sequence length is longer than the specified maximum sequence length for this model (3008 > 512). Running this sequence through the model will result in indexing errors


dict_keys(['text', 'meta', 'input_ids', 'attention_mask'])

In [29]:
# interleave_datasets로 결합하기
wrapper.combined_dataset = interleave_datasets(
    [wrapper.pile_dataset["train"].take(10),
     wrapper.pile_dataset["test"].take(10)]
)
print(list(islice(wrapper.combined_dataset, 2)))

del compare
wrapper.init_wrapper()

[{'text': 'It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.\n\nThere’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.\n\nConcept\n\nWorking over the theme was probably one of the hardest tasks I had to face.\n\nOriginally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.\n\nIn the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a species facing a challenge